In [0]:
from pyspark.sql.functions import *

In [0]:
aterio = spark.sql('''SELECT * FROM `aterio_io_us_real_estate_investment_ratings_by_zipcode`.`delievery`.`us_aterio_score_zipcode`''')
real_estate = spark.sql('''SELECT * FROM bright_data_real_estate_listings.datasets.zillow_properties''')
nimble = spark.sql('''SELECT * FROM `nimble_international_residential_commercial_real_estate_data_pipelines`.`nimble_ret`.`us_listings_daily`
''')

In [0]:
real_estate_curated = real_estate.withColumn('LASTSOLDPRICE', when(col('LASTSOLDPRICE') == 'null', lit(None)).otherwise(col('LASTSOLDPRICE')))\
.withColumn('PRICE', when(col('PRICE') == 0, None).otherwise(col('PRICE')))\
.withColumn('DESCRIPTION', when(col('DESCRIPTION') == 'null', lit(None)).otherwise(col('DESCRIPTION')))\
.withColumn('URL', when(col('URL') == 'null', lit(None)).otherwise(col('URL')))\
.select(
  'HOMESTATUS',
  'BEDROOMS',
  'BATHROOMS',
  'PRICE',
  'YEARBUILT',
  'STREETADDRESS',
  'ZIPCODE',
  'LONGITUDE',
  'LATITUDE',
  'HOMETYPE',
  'LIVINGAREAVALUE',
  'LIVINGAREAUNITSSHORT',
  'RENTZESTIMATE',
  'CURRENCY',
  'COUNTRY',
  'LASTSOLDPRICE',
  'DESCRIPTION',
  'URL'
).distinct()

In [0]:
#normalize arterio risk factor
min_max_df = aterio.select(min("IDX_RISK_FACTOR").alias("min_risk"), max("IDX_RISK_FACTOR").alias("max_risk"))
min_risk, max_risk = min_max_df.first()
aterio = aterio.withColumn("normalized_risk_factor", (col("IDX_RISK_FACTOR") - min_risk) / (max_risk - min_risk))
risk_factor = aterio.select(
  col('ZIP_CODE'),
  col('CITY_NAME'),
  col('STATE_CODE'),
  col('ATERIO_SCORE'),
  col('normalized_risk_factor').alias('risk_factor')
).distinct()

In [0]:

final_data_set = risk_factor.alias('rsk').join(real_estate_curated.alias('rel'), col('rsk.ZIP_CODE') == col('rel.ZIPCODE'))\
  .select(
    col('ZIP_CODE'),
    col('CITY_NAME'),
    col('STATE_CODE'),
    col('LONGITUDE'),
    col('LATITUDE'),
    col('STREETADDRESS'),
    col('ATERIO_SCORE'),
    col('risk_factor'),
    col('HOMESTATUS'),
    col('BEDROOMS'),
    col('BATHROOMS'),
    col('PRICE'),
    col('YEARBUILT'),
    col('HOMETYPE'),
    col('LIVINGAREAVALUE'),
    col('LIVINGAREAUNITSSHORT'),
    col('RENTZESTIMATE'),
    col('CURRENCY'),
    col('COUNTRY'),
    col('LASTSOLDPRICE'),
    col('DESCRIPTION'),
    col('URL')
  ).distinct()

In [0]:
final_data_set.display()

ZIP_CODE,CITY_NAME,STATE_CODE,LONGITUDE,LATITUDE,STREETADDRESS,ATERIO_SCORE,risk_factor,HOMESTATUS,BEDROOMS,BATHROOMS,PRICE,YEARBUILT,HOMETYPE,LIVINGAREAVALUE,LIVINGAREAUNITSSHORT,RENTZESTIMATE,CURRENCY,COUNTRY,LASTSOLDPRICE,DESCRIPTION,URL
90740,Seal Beach,CA,-118.08336,33.765366,13681 Saint Andrews Dr #1-25I,565.56125216,0.27846878547051823,RECENTLY_SOLD,1,1,229000,1962,CONDO,600,sqft,2152,USD,USA,229000,"Great Location! Pretty View! Lots of Trees! Enhance the desirability of this one Bedroom one bathroom cottage with a terrific Greenbelt view. Move in and make this property your own. The spacious Living Room has a heat pump and air conditioning. The Kitchen has built-in Leisure World appliances including a stove, oven, refrigerator and a skylight that opens. Bedroom is carpeted and has a wardrobe closet. The Bathroom has a cut-down shower for walk-in convenience and a skylight with dome that opens. Located in Leisure World Seal Beach a resort like community with over 200 clubs, organizations and activities, including a gym, 9 hole 3 par golf course, swimming pool, jacuzzi, amphitheater, Bocce ball, Shuffle board, Wood working, sewing, arts and crafts, Library, Healthcare and so much more. This property is located in the unique beach town of Seal Beach. Move-in and enjoy the delightful ocean breezes.",https://www.zillow.com/homedetails/13681-Saint-Andrews-Dr-1-25I-Seal-Beach-CA-90740/2055440589_zpid/
78941,Flatonia,TX,-97.11116,29.684015,417 S Faires St,529.10262106,0.4881377180926714,FOR_SALE,3,2,269500,1994,SINGLE_FAMILY,1596,sqft,1999,USD,USA,null,"New Composition Roof, new exterior paint and beautiful new landscaping dress this brick 3 BR, 2 Bath home in Flatonia, The property has attractive curb appeal with the red brick, white trim and black front door. Mature trees shade the yard and home. The entry area welcomes you to the spacious living area with wood burning fire place. The dinning area separates the family room and kitchen. The kitchen has abundant light oak wooden cabinets, plenty of counter space and a pantry. Most rooms have ceilings fans and attractive vinyl plank flooring for easy care. Dressing up the interior of the home are white plantation wooden shades. The primary bedroom is large enough for a king size bed, if desired; and the second and third bedrooms are a generous size as well. An attached 2 car garage has room for extra storage or appliances and contains the washer and dryer hook ups. The home is surrounded with mature trees and you will enjoy the fenced back yard with deck and storage building. There is ample room for gardening or outdoor entertaining. This home has much to offer and is waiting for someone to come in and make it their own. It will provide comfortable living in a wonderful Central Texas community located halfway between Houston, San Antonio and Austin. Just 35 minutes to Bastrop. Walking distance to the school and downtown Flatonia. New roof has 30 yr Warranty on shingles.",https://www.zillow.com/homedetails/417-S-Faires-St-Flatonia-TX-78941/2078157680_zpid/
48906,Lansing,MI,-84.538124,42.7451,1017 N Pennsylvania Ave,600.44137087,0.30924748959415405,FOR_SALE,3,1,39400,1916,SINGLE_FAMILY,1365,sqft,1400,USD,USA,30000,"Classic American Four Square with 2 car garage. Mechanicals, plumbing, and electrical have been approved by city. Located near Sparrow, Downtown, and Old Town.",https://www.zillow.com/homedetails/1017-N-Pennsylvania-Ave-Lansing-MI-48906/127050495_zpid/
77380,Spring,TX,-95.464165,30.118341,47 Camberwell Ct,748.96872028,0.2686708937886527,FOR_SALE,2,2,305000,1983,SINGLE_FAMILY,1365,sqft,1849,USD,USA,null,"Look no further, come quick to view this darling one story patio home tucked into a private wooded lot at the end of a cul-de-sac, backing to a greenbelt. Split bedroom floorplan offers 2 primary bedrooms & private baths each with two sinks, plus a study that could be converted to a third bedroom. Light, bright and open floorplan with high vaulted ceiling, tile floors, 

In [0]:
final_data_set.write.format("delta").saveAsTable("default.us_listings_with_risk_scores")

---

In [0]:
cleaned_us_listings = spark.sql('''SELECT * FROM workspace.default.us_listings_daily_cleaned''')

In [0]:
clean_us_listings_joined = risk_factor.alias('rsk').join(cleaned_us_listings.alias('cl'), col('rsk.ZIP_CODE') == col('cl.zip')).select(
  'rsk.*',
  'address',
  'agent_business_name',
  'agent_info',
  'agent_name',
  'agent_type',
  'applications',
  'baths',
  'beds',
  'city',
  'contacts',
  'days_listed',
  'description',
  'home_status',
  'home_type',
  'is_building',
  'is_complex',
  'lat',
  'lng',
  'maybe_sold',
  'original_address',
  'owner',
  'rent',
  'skipped',
  'sqft',
  'state',
  'street',
  'zip',
  'snapshot_date',
  'distance',
  'name',
  'rating',
  'level',
  'studentsPerTeacher',
  'assigned',
  'grades',
  'link',
  'type',
  'size',
  'totalCount',
  'isAssigned',
  'date',
  'time',
  'price',
  'pricePerSquareFoot',
  'priceChangeRate',
  'event',
  'source',
  'buyerAgent',
  'sellerAgent',
  'showCountyLink',
  'postingIsRental'
).distinct()


In [0]:
final_data_set.write.format("delta").saveAsTable("default.southeast_us_listings_with_risk_scores")